教授にプレゼン後、データの保存をし、テーブルとして値を持つ必要があることがわかった
大きなデータになるので保存方法を考慮する必要があった。
CSV、Excel、バイナリ、HDF5が候補として挙がった。
HDF5形式は大容量のデータセットを効率的に保存するためのファイル形式であるため、採択。
参考サイトの少なさから一時的にSQLを採択
SQLをテストし、全語句のデータの保存をしてみた
SQLでは、VisualStudioCodeの拡張機能が使いづらく、SQLite3に変更した
SQLite->dataframe->変更->dataframe->変更を保存->SQLiteの完成
上記の方法では変更件数に関わらず、全件で1分30秒かかる。
変更に時間がかかりすぎな点が課題であるため、データの更新方法を検討
SQLからPythonに値を渡さず、直接書き込むことで一件当たり0.1秒以内

In [2]:
import pandas as pd
from gensim.models import Word2Vec
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import japanize_matplotlib
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# 学習済みモデルのロード
model_file_path = '../../KITERETU/gw2v160.model'
model = Word2Vec.load(model_file_path)

In [5]:
from gensim.models import Word2Vec
import sqlite3

# データベースに接続し、カーソルを作成する
conn = sqlite3.connect('database2.sqlite')
cursor = conn.cursor()

# テーブルが存在しない場合のみ作成する
cursor.execute('''CREATE TABLE IF NOT EXISTS words (word TEXT, value REAL)''')

# モデル内のすべての単語と値をデータベースに保存する
for word in model.wv.key_to_index:
    cursor.execute('''INSERT INTO words VALUES (?, ?)''', (word, 0.000000))

# コミットして変更を確定する
conn.commit()

# データベース接続を閉じる
conn.close()


In [6]:
import sqlite3

# データベースに接続する
conn = sqlite3.connect('database2.sqlite')
cursor = conn.cursor()

# データの更新を行う
cursor.execute("UPDATE words SET value = ? WHERE word = ?", (0.20000, '其の'))

# コミットして変更を確定する
conn.commit()

# データベース接続を閉じる
conn.close()


In [5]:
input_str=input("意味類似度ネットワークを作りたい単語を入力してください===>")


群馬に類似する文字列とその類似度は以下の通り
群馬県 	 0.7760874032974243
栃木 	 0.7456161379814148
前橋 	 0.7389767169952393
埼玉 	 0.7216979265213013
高崎 	 0.6891007423400879
伊勢崎 	 0.6693983674049377
茨城 	 0.6651455163955688
高崎市 	 0.6536263227462769
群馬県前橋市 	 0.6474608778953552
栃木県 	 0.6421990990638733


In [ ]:
# 似ている単語TOP7を書き出す
out = model.wv.most_similar(positive=[input_str], topn=3)

#AtCoderでよく使う辞書処理を使って県と数値を取り出す
pre=[]
pre.append(input_str)
for prefecture, value in out:
    print(prefecture,"\t",value)
    pre.append(prefecture)


In [7]:
input_str=input("意味類似度ネットワークを作りたい単語を入力してください===>")

# dataframeをいったん作成する
data = {
    'input': ['群馬県', '栃木'],
    'similar_words': [[0.7760874032974243], [0.7456161379814148]]
}
df = pd.DataFrame(data)

for i in range(3):
    # 似ている単語TOP7を書き出す
    out = model.wv.most_similar(positive=[input_str], topn=3)

    #AtCoderでよく使う辞書処理を使って県と数値を取り出す
    pre=[]
    pre.append(input_str)
    for prefecture, value in out:
        print(prefecture,"\t",value)
        pre.append(prefecture)    
        # キーが既存の場合は値を加算し、存在しない場合は新しい行をDataFrameに追加する
        # prefectureでキーが存在するか確認
        # 存在した場合は値を追加
        # 存在しなかった場合は新たにdfに追加する

群馬県 	 0.7760874032974243
栃木 	 0.7456161379814148
前橋 	 0.7389767169952393
群馬県 	 0.7760874032974243
栃木 	 0.7456161379814148
前橋 	 0.7389767169952393
群馬県 	 0.7760874032974243
栃木 	 0.7456161379814148
前橋 	 0.7389767169952393


In [18]:
import pandas as pd

input_str = input("意味類似度ネットワークを作りたい単語を入力してください===>")

# dataframeをいったん作成する
data = [
    [['群馬県'], [0.7760874032974243]],
    [['栃木'], [0.7456161379814148]]
]
df = pd.DataFrame(data)

for i in range(3):
    # 似ている単語TOP7を書き出す
    out = model.wv.most_similar(positive=[input_str], topn=3)

    # AtCoderでよく使う辞書処理を使って県と数値を取り出す
    similar_words = [input_str]
    for prefecture, value in out:
        print(prefecture, "\t", value)
        similar_words.append(prefecture)

        # キーが既存の場合は値を加算し、存在しない場合は新しい行をDataFrameに追加する
        if any((df[0] == prefecture).all(axis=1)):
            existing_indices = df[df[0] == prefecture].index
            existing_row = df.loc[existing_indices[0]]
            existing_row[1].extend([value])
            df.loc[existing_indices[0]] = existing_row
        else:
            new_row = pd.DataFrame([[prefecture], [[value]]])
            df = pd.concat([df, new_row], ignore_index=True)

print(df)


群馬県 	 0.7760874032974243


ValueError: No axis named 1 for object type Series